## 1. Prepare the data. 
20 news groups data is one of the standard datasets in the scikit-learn. 

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Explore the data.

In [2]:
from numpy.core.fromnumeric import sort
# what is the data type of train and test? What is the data type of train['data']? What is in train['target_names']

# train = ['data', 'filenames', 'target_names', 'target', 'DESCR']
# test = ['data', 'filenames', 'target_names', 'target', 'DESCR']
# type(train['data']) = list; 11314
# len(train['target_names']) = 20
# train['target_names'] = 0..19


# print(type(train['target_names']), type(test)) # Bunch-dictionary object
#print(test.keys())
#print(sort(train['target']))
#print(len(train['target_names'])) # list of str 

In [19]:
train['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [20]:
train['data'][0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

Train / validation / test splits.

In [3]:
from sklearn.model_selection import train_test_split 
import pandas as pd

X, y = pd.Series(train.data), pd.Series(train.target)
X_test, y_test = pd.Series(test.data), pd.Series(test.target)

X_train, X_valid, y_train, y_valid = train_test_split(X,y, random_state=19, test_size=0.1) 

assert y_train.shape == (10182,)
assert y_valid.shape == (1132,)
assert y_test.shape == (7532,)

In [22]:
y_train

6460     14
7945     14
9538     18
5782      9
8464      3
         ..
1043      3
5032     12
1378      5
757       7
10862     5
Length: 10182, dtype: int64

## 2. Model (WHY?)

 the natural language processing tasks. 

 When working on a supervised machine learning problem with a given data set, we try different algorithms and techniques to search for models to produce general hypotheses, which then make the most accurate predictions possible about future instances.

In [23]:
#  Logistic Regression, which provides a nice baseline

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test)) # 83%

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


accuracy 0.8317843866171004


## Pre-trained Transformer model DistilBERT 

In [20]:
# Convert categorical labels into dummy variables. 
y_train = pd.get_dummies(y_train)
y_valid = pd.get_dummies(y_valid)
y_test = pd.get_dummies(y_test)

NUM_LABELS = 20 # len(train['target']))
assert y_train.shape == (10182, NUM_LABELS)
assert y_valid.shape == (1132, NUM_LABELS)
assert y_test.shape == (7532, NUM_LABELS)

In [21]:
from keras.preprocessing.text import Tokenizer

VOCAB_SIZE = 10_000
MAX_LEN = 256
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<unk>')
tokenizer.fit_on_texts(X_train)

#from keras.preprocessing import sequence
from keras.utils import pad_sequences

def preprocess(texts, tokenizer, maxlen:int = MAX_LEN):
    seqs = tokenizer.texts_to_sequences(texts) 
    tokenized_text = pad_sequences(seqs, maxlen=MAX_LEN) 
    return tokenized_text

X_train = preprocess(X_train, tokenizer, MAX_LEN)
X_valid = preprocess(X_valid, tokenizer, MAX_LEN)
X_test  = preprocess(X_test, tokenizer, MAX_LEN) 

In [25]:
# Transformer model from Hugging Face 
# DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter, distillbert-uncased specifically, and fine-tune it on the 20 Newsgroups dataset.

In [5]:
! pip install -U -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.8 MB/s eta 0:00:00


In [27]:
import transformers
print(transformers.__version__)

4.27.1


In [37]:
# Create Dataset objects for train / validation / test sets that are better compatible with the Transformers API.

from datasets import Dataset
train_ds = Dataset.from_pandas(pd.DataFrame({'text': pd.Series(train.data), 'label':pd.Series(train.target)}))
test_ds  = Dataset.from_pandas(pd.DataFrame({'text': pd.Series(test.data), 'label':pd.Series(test.target)}))

train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 11314
})

In [38]:
train_ds = train_ds.class_encode_column('label')
test_ds = test_ds.class_encode_column('label')

train_ds.features

Stringifying the column:   0%|          | 0/11314 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/11314 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/7532 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/7532 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '3', '4', '5', '6', '7', '8', '9'], id=None)}

In [39]:
# Create the validation set
from datasets.utils import stratify
train_dsd = train_ds.train_test_split(test_size=0.1, stratify_by_column='label', seed=19)
train_dsd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1132
    })
})

In [40]:
train_dsd['validation'] = train_dsd['test']
train_dsd['test'] = test_ds

In [41]:
train_dsd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1132
    })
})

 Load the DistilBERT tokenizer to process the text. 

In [42]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [43]:
# a preprocessing function to tokenize text
VOCAB_SIZE = 10_000
EMBED_DIM = 256
DENSE_DIM = 32
NUM_HEADS = 2
MAX_LEN = 256

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=MAX_LEN)

In [44]:
# apply the preprocessing function over the entire dataset. You can speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_text = train_dsd.map(preprocess_function, batched=True)
tokenized_text

Map:   0%|          | 0/10182 [00:00<?, ? examples/s]

Map:   0%|          | 0/7532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1132
    })
})

In [45]:
#  a batch of examples
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf') 

In [46]:
# To fine-tune a model in TensorFlow, start by converting datasets to the tf.data.Dataset format with to_tf_dataset 
BATCH_SIZE = 16

tf_train_set = tokenized_text["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)
tf_validation_set = tokenized_text["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True, 
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )
tf_test_set = tokenized_text["test"].to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle= True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [47]:
from transformers import create_optimizer

EPOCHS = 5
batches_per_epoch = len(tokenized_text["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch * EPOCHS)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [48]:
# Load DistilBERT with TFAutoModelForSequenceClassification along with the number of expected labels

from transformers import TFAutoModelForSequenceClassification

my_bert = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=20)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [49]:
my_bert.compile(optimizer=optimizer,  metrics=['accuracy']) #  for training

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [50]:
# fine-tuning
my_bert.fit(x=tf_train_set, epochs=2, batch_size=BATCH_SIZE)

Epoch 1/2
637/637 [==============================] - 288s 418ms/step - loss: 1.3190 - accuracy: 0.6821
Epoch 2/2
637/637 [==============================] - 267s 419ms/step - loss: 0.4280 - accuracy: 0.8820


In [51]:
# Evaluate the model on the test data
bert_loss, bert_acc = my_bert.evaluate(tf_test_set) # YOUR CODE HERE
bert_acc # 82% 

471/471 [==============================] - 70s 146ms/step - loss: 0.5851 - accuracy: 0.8281


0.828066885471344